In [2]:
# import ray
# ray.shutdown()

# import pandas as pd
import modin.pandas as pd
import pickle
import random
from tqdm import tqdm
import os

In [3]:
with open('pickles/fasta_dict.pkl', 'rb') as f:
    fasta_records = pickle.load(f)
with open('pickles/clusters.pkl', 'rb') as f:
    clusters = pickle.load(f)
print('Proteins in clusters: {}'.format(len(clusters)))
print('Number of clusters: {}'.format(len(set(clusters.values()))))

Proteins in clusters: 7709111
Number of clusters: 498526


In [11]:
interactions_raw = pd.read_csv('string12.0_combined_score_900.tsv', sep=' ', usecols=['protein1', 'protein2', 'combined_score'])
interactions_raw['clusters'] = interactions_raw.apply(lambda row: (clusters[row['protein1']], clusters[row['protein2']]), axis=1)
interactions = interactions_raw[interactions_raw['combined_score'] > 900]
interactions['score'] = 1
interactions = interactions.drop(columns='combined_score')
# interactions.to_csv('positives.tsv', sep='\t', index=False, header=False)

In [4]:
interactions = pd.read_csv('positives.tsv', sep='\t', header=None, names=['protein1', 'protein2', 'clusters', 'score'])
print(interactions.head())
unique_clusters = set(interactions['clusters'].values)
interactions = interactions.drop(columns='clusters')

2024-08-20 18:12:50,260	INFO worker.py:1781 -- Started a local Ray instance.
(raylet) /Users/volzhenin/PycharmProjects/T5-ppi/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
(raylet)   warnings.warn(


         protein1        protein2  \
0  23.BEL05_00045  23.BEL05_15465   
1  23.BEL05_00045  23.BEL05_07675   
2  23.BEL05_00090  23.BEL05_00095   
3  23.BEL05_00095  23.BEL05_00090   
4  23.BEL05_00145  23.BEL05_13290   

                                            clusters  score  
0          ('1492922.GY26_20280', '28176.CF66_7091')      1  
1         ('1492922.GY26_20280', '67801.A0A1B0C7F8')      1  
2     ('1798274.A3E87_01195', '1262974.BN779_00805')      1  
3     ('1262974.BN779_00805', '1798274.A3E87_01195')      1  
4  ('286419.ENSCAFP00020032892', '278944.A0A4Z1IE...      1  


(raylet) Spilled 3734 MiB, 13 objects, write throughput 1991 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 5237 MiB, 18 objects, write throughput 2160 MiB/s.
(raylet) /Users/volzhenin/PycharmProjects/T5-ppi/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020 [repeated 7x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(raylet)   warnings.warn( [repeated 7x across cluster]


In [5]:
proteins_dict1 = interactions['protein1'].value_counts().to_dict()
proteins_dict2 = interactions['protein2'].value_counts().to_dict()
proteins_dict = {k: proteins_dict1.get(k, 0) + proteins_dict2.get(k, 0) for k in set(proteins_dict1) | set(proteins_dict2)}
# proteins_list = set(interactions['protein1'].values) | set(interactions['protein2'].values)

the groupby keys will be sorted anyway, although the 'sort=False' was passed. See the following issue for more details: https://github.com/modin-project/modin/issues/3571.
Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [6]:
def compare_to_unique(row):
    cluster1 = clusters[row['protein1']]
    cluster2 = clusters[row['protein2']]

    return (cluster1, cluster2) in unique_clusters or (cluster2, cluster1) in unique_clusters

In [9]:
positive_len = len(interactions)
file_len = 0
if os.path.exists('negatives.tsv'):
    with open('negatives.tsv', 'r') as f:
        for line in tqdm(f):
            file_len += 1
print('Initial file length: {}'.format(file_len))
print('Required file length: {}'.format(positive_len * 10))
print('Pairs left: {}'.format(positive_len * 10 - file_len))

prots = list(proteins_dict.keys())
weights = proteins_dict.values()

chuncksize = 20000000
while file_len < positive_len * 10:
    negative_pairs = pd.DataFrame({'protein1': random.choices(prots, weights=weights, k=chuncksize), 'protein2': random.choices(prots, weights=weights, k=chuncksize)})
    negative_pairs = negative_pairs[~negative_pairs.apply(lambda row: compare_to_unique(row), axis=1)]
    negative_pairs['score'] = 0
    negative_pairs.to_csv('negatives.tsv', sep='\t', index=False, header=False, mode='a')
    file_len += len(negative_pairs)
    print(file_len)

40300000it [00:10, 3803851.48it/s]


Initial file length: 40300000
Required file length: 723379960
Pairs left: 683079960


60300000


KeyboardInterrupt: 

In [ ]:
negatives = pd.read_csv('negatives.tsv', sep='\t', header=None, names=['protein1', 'protein2', 'score'])

In [5]:
# positive_len = 72337996
with open('all_900.tsv', 'w') as f:
    with open('positives.tsv', 'r') as fr:
        for line in tqdm(fr):
            args = line.strip().split('\t')
            f.write('\t'.join([args[0], args[1], args[-1]])+'\n')
    leng = 0
    with open('negatives.tsv', 'r') as fr:
        for line in tqdm(fr):
            f.write(line.strip()+'\n')
            leng += 1
            if leng > positive_len * 10:
                break

72337996it [00:46, 1557749.41it/s]
732400000it [04:06, 2971483.85it/s]


In [7]:
with open('all_900.tsv', 'r') as f:
    with open('all_900_train.tsv', 'w') as f_train:
        with open('all_900_test.tsv', 'w') as f_test:
            for line in tqdm(f):
                if random.random() < 1000000 / positive_len / 11:
                    f_test.write(line)
                else:
                    f_train.write(line)

794718562it [02:05, 6356001.51it/s]
